In [2]:
import random
import os
import re
import numpy as np
from pprint import pprint
print(os.getcwd())
from Backend.py_utils import *
from Backend import perf_calc as perf_calc
from fastapi import FastAPI, Request
from fastapi.responses import RedirectResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from Backend import perf_calc as perf_calc
from Backend import rotation_calc as rotation_calc
from Backend import refusal as refusal
from database import database_handler
from fastapi.responses import JSONResponse
from Backend import handle_route as handle_route
from pydantic import BaseModel 
from Backend import wind_calc
import sqlite3

g:\CompSciProject
g:\CompSciProject
g:\CompSciProject
g:\CompSciProject


In [11]:
def search_airport(partial_name):
    database_path = "database/nav_data.db"
    connect_to_db = sqlite3.connect(database_path)
    cursor = connect_to_db.cursor()

    words = partial_name.split()
    conditions = " AND ".join(["name LIKE ? COLLATE NOCASE" for _ in words])

    query = f"""
    SELECT name, icao,type
    FROM airports 
    WHERE {conditions} 
    ORDER BY 
        LENGTH(name), 
        INSTR(LOWER(name), LOWER(?)) 
    """

    params = tuple(f"%{word}%" for word in words) + (words[0],)

    cursor.execute(query, params)
    results = cursor.fetchall()
    connect_to_db.close()

    return results
search_airport("hello")

[('Othello Municipal Airport', 'KS70', 'small_airport')]

In [ ]:
from math import cos, asin, sqrt, pi
import sqlite3
def nearby_points(point_lat, point_lng, area_code):
    database_path = "database/nav_data.db"
    connect_to_db = sqlite3.connect(database_path)
    cursor = connect_to_db.cursor()

    cursor.execute("SELECT waypoint_identifier, waypoint_latitude, waypoint_longitude FROM waypoints WHERE area_code = ?", (area_code,))

    close_wp = []

    data = cursor.fetchall()

    def distance(lat1, lon1, lat2, lon2):
        r = 3958.8 # miles
        p = pi / 180

        a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
        return 2 * r * asin(sqrt(a))
    
    for waypoint in data:
        waypoint_id, waypoint_lat, waypoint_lng = waypoint
        dist = distance(point_lat, point_lng, waypoint_lat, waypoint_lng)
       
        if dist < 20:
            close_wp.append([waypoint_id,waypoint_lat,waypoint_lng, dist])
    
    return close_wp

nearby_points(51.68527778,0.68333333,"EUR")